In [2]:
import pickle  # To load the saved SVM model and TF-IDF vectorizer
import numpy as np  # For numerical operations
from scipy.sparse import csr_matrix  # For working with sparse matrices (used in TF-IDF)
from sklearn.svm import SVC  # For support vector classifier (if not already imported during training)

In [6]:
def custom_bangla_tokenizer(text):
    return text.split() 

# Load the TF-IDF vectorizer with the custom tokenizer
with open(r'C:\Users\mdnah\OneDrive\Desktop\TextClassification\TFIDF_Vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

# Load the SVM model
with open(r'C:\Users\mdnah\OneDrive\Desktop\TextClassification\SVM_Model.pkl', 'rb') as f:
    rf_model = pickle.load(f)

# Input sentence for prediction
input_sentence = "শীতের ছুটির জন্য একটা বড় প্ল্যান করছি এবার।"

# Transform the input sentence using the TF-IDF vectorizer
input_vector = tfidf_vectorizer.transform([input_sentence])

# Predict the class using the SVM model
y_pred_rf = rf_model.predict(input_vector)

# Reverse label mapping
class_mapping = {0: "Dementia", 1: "Non-Dementia"}
predicted_class = class_mapping[y_pred_rf[0]]

# Output the prediction
print(f"Predicted Class: {predicted_class}")

Predicted Class: Non-Dementia


In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [7]:
# Load the saved BiLSTM model
model_save_path = r'C:\Users\AsifAK\Desktop\Code_Detect_Dementia\SenComplete\BiLSTM_model.h5'
model = load_model(model_save_path)
#print(f"Model loaded from: {model_save_path}")

# Load the saved tokenizer
tokenizer_save_path = r"C:\Users\AsifAK\Desktop\Code_Detect_Dementia\SenComplete\sac_tokenizer.pkl"
with open(tokenizer_save_path, 'rb') as file:
    tokenizer = pickle.load(file)
#print("Tokenizer loaded successfully.")

# Function to complete the sentence
def complete_sentence_with_end_token(model, tokenizer, seed_text, max_sequence_len, max_words=10):
    completed_sentence = seed_text
    for _ in range(max_words):
        token_list = tokenizer.texts_to_sequences([completed_sentence])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs, axis=-1)
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break

        # Stop if the word is the <end> token
        if output_word == "end":
            break

        # Append the predicted word to the sentence
        completed_sentence += " " + output_word

        completed_sentence = completed_sentence.replace(" end", "")

    return completed_sentence

# Initial seed text
seed_text = "আমার রুটি দিয়ে হাঁসের"  # Replace with your Bangla text সকালে আমি নাস্তা করেছিলাম  এবার করবানির ছাগলের    আমার রুটি দিয়ে হাঁসের  এই জামাটা পরতে

# Define the number of words to predict
#num_words_to_predict = 5 # Adjust based on your desired completion length
max_sequence_len = 15
# Complete the sentence
completed_sentence = complete_sentence_with_end_token(model, tokenizer, seed_text, max_sequence_len)
print(f"Completed Sentence: {completed_sentence}")

Completed Sentence: আমার রুটি দিয়ে হাঁসের মাংস খেতে ইচ্ছা করছে


In [ ]:
import os
import numpy as np
import librosa
import joblib
import noisereduce as nr  # Import the noise reduction library
from flask import Flask, request, jsonify, render_template
from tensorflow.keras.models import load_model  # type: ignore

# Initialize Flask app
app = Flask(__name__)

# Paths to your model and scaler
MODEL_PATH = r'C:\Users\AsifAK\Desktop\TextDementia\Dementia website - Edited\models\lstm_model.h5'
SCALER_PATH = r'C:\Users\AsifAK\Desktop\TextDementia\Dementia website - Edited\models\scaler.pkl'

# Load the model and scaler
model = load_model(MODEL_PATH)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
scaler = joblib.load(SCALER_PATH)

# Ensure uploads folder exists
if not os.path.exists('uploads'):
    os.makedirs('uploads')

# Function to extract features from the audio file with noise reduction
def extract_features(audio_path):
    try:
        # Load the audio file
        y, sr = librosa.load(audio_path, sr=None)

        # Apply noise reduction
        y_denoised = nr.reduce_noise(y=y, sr=sr)  # Reduce noise from the audio

        # Extract MFCCs (Mel Frequency Cepstral Coefficients)
        mfcc = np.mean(librosa.feature.mfcc(y=y_denoised, sr=sr, n_mfcc=13), axis=1)

        # Extract Chroma (Chroma features are related to pitch)
        chroma = np.mean(librosa.feature.chroma_stft(y=y_denoised, sr=sr), axis=1)

        # Extract Spectral Centroid
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y_denoised, sr=sr))

        # Extract Spectral Bandwidth
        spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y_denoised, sr=sr))

        # Extract Zero-Crossing Rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=y_denoised))

        # Extract Root Mean Square (RMS) energy
        rms = np.mean(librosa.feature.rms(y=y_denoised))

        # Extract Jitter (Pitch variation)
        pitches, _ = librosa.piptrack(y=y_denoised, sr=sr)
        pitch_values = pitches[pitches > 0]
        jitter = np.mean(np.abs(np.diff(pitch_values)) / pitch_values[:-1]) if len(pitch_values) > 1 else 0

        # Combine all the features into a single feature vector
        feature_vector = np.hstack([mfcc, chroma, spectral_centroid, spectral_bandwidth, 
                                    zero_crossing_rate, rms, jitter])

        # Reshape the feature vector into a 3D array: (1, 1, 30)
        feature_vector_reshaped = np.expand_dims(feature_vector, axis=0)  # Shape: (1, 30)
        feature_vector_reshaped = np.expand_dims(feature_vector_reshaped, axis=1)  # Shape: (1, 1, 30)

        # Scale the features using the scaler
        feature_vector_scaled = scaler.transform(feature_vector_reshaped.reshape(1, -1))  # Flatten for scaling
        feature_vector_scaled = feature_vector_scaled.reshape(1, 1, -1)  # Reshape back to (1, 1, 30)

        return feature_vector_scaled
    except Exception as e:
        print(f"Error extracting features: {e}")
        raise

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if 'file' not in request.files:
            return jsonify({'error': 'No file part'}), 400

        file = request.files['file']

        if file.filename == '':
            return jsonify({'error': 'No selected file'}), 400

        if file and file.filename.endswith('.wav'):
            file_path = os.path.join('uploads', file.filename)
            file.save(file_path)

            # Debug: Log file details
            print(f"Received file: {file.filename}")
            print(f"File path: {file_path}")

            features = extract_features(file_path)
            prediction = model.predict(features)

            print(f"Prediction result: {prediction}")

            prediction_label = 'Dementia' if prediction[0][0] > 0.5 else 'Non-Dementia'
            return jsonify({'prediction': prediction_label}), 200
        else:
            return jsonify({'error': 'Invalid file format, only .wav allowed'}), 400
    except Exception as e:
        print(f"Error during prediction: {e}")
        return jsonify({'error': 'An error occurred during prediction'}), 500

if __name__ == '__main__':
    app.run(debug=True)